# Set Required Args to Activate Modules

In [ ]:
# from workflow.scripts.resources import Resources

# single resource

In [ ]:
# # Iterate over provinces for both solar and wind resources
# resource_types = ['wind', 'solar']
# provinces=['BC']  #'AB','SK','ON','NS'
# for province_code in provinces:
#     for resource_type in resource_types:
#         required_args = {
#             "config_file_path": 'config/config.yml',
#             "province_short_code": province_code,
#             "resource_type": resource_type
#         }
        
#         # Create an instance of Resources and execute the module
#         resource_module = Resources(**required_args)
#         resource_module.execute_module()
        

In [2]:
import geopandas as gpd
import pandas as pd
from linkingtool.hdf5_handler import DataHandler
store=DataHandler('data/store/resources_BC.h5')
from linkingtool.gwa import GWACells
gwa=GWACells(resource_type='wind')

>> Store initialized with the given path: data/store/resources_BC.h5
>> Store initialized with the given path: data/store/resources_BC.h5


In [3]:
cells=store.from_store('cells')

In [4]:
data=gwa.load_gwa_cells()

2024-11-03 11:08:37,678 - INFO - >> Loading GADM boundaries (Sub-provincial | level =2) for British Columbia  from local file data/processed_data/regions/gadm41_Canada_L2_BC.geojson.
2024-11-03 11:08:43,029 - INFO - Memory resource limitations enabled. Filtering GWA cells witn windspeed mask to limit the data offload processing...
2024-11-03 11:08:43,320 - INFO - >> 24530090 cells have been filtered due to Windspeed filter [7-30 m/s].
>>> Cleaned data loaded for 10003944 GWA cells
2024-11-03 11:08:43,323 - INFO - >> Global Wind Atlas (GWA) Cells loaded. Size: 10003944
2024-11-03 11:08:45,412 - INFO - >> Loading GADM boundaries (Sub-provincial | level =2) for British Columbia  from local file data/processed_data/regions/gadm41_Canada_L2_BC.geojson.


In [ ]:
results = []  # List to store results for each region

for region in cells['Region'].unique():
    _store_grid_cells_region = cells[cells['Region'] == region]
    
    # Perform overlay operation
    _data_ = gpd.overlay(data, _store_grid_cells_region, how='intersection', keep_geom_type=False)
    
    # Rename columns and select relevant data
    _data_ = _data_.rename(columns={'x_1': 'x', 'y_1': 'y'})
    selected_columns = list(_data_.columns) + [f'wind_CF_mean']
    
    # Store mapped GWA cells in results list
    results.append(_data_.loc[:, selected_columns])

# Concatenate all results into a single Dask GeoDataFrame
if results:
    mapped_gwa_cells = pd.concat(results, axis=0).drop_duplicates()
    
    # self.log.info(f">> Calculating aggregated values for ERA5 Cell's...")
    
    # Aggregate values using Dask
    mapped_gwa_cells_aggr = mapped_gwa_cells.groupby('cell').agg({
        'windspeed_gwa': 'mean',
        'CF_IEC2': 'mean',
        'CF_IEC3': 'mean',
        'wind_CF_mean': 'mean'
    }, numeric_only=True)
    
    # Store the aggregated data
    store.to_store(mapped_gwa_cells_aggr, 'cells')  # Compute and store results